In [ ]:
library(INLA) ##
library(MASS)
library(rSPDE)
library(readr)

## generate mesh 

In [ ]:
data <- as.data.frame(read_csv("1180.SR.csv"))
logpt<-log(data$Close)#
logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
date <- data$Date[1:(length(data$Date)-1)]
dates<-as.integer(format(as.Date(date),"%Y%M%d"))


## implement the model& fit model 

In [ ]:
mesh_loc <- inla.mesh.1d(dates) # 

Abar <- rspde.make.A(mesh = mesh_loc, loc = dates)

rspde_model <- rspde.matern(mesh = mesh_loc,rspde.order=1)
mesh.index <- rspde.make.index(name = "field", mesh = mesh_loc)
data = list(logreturn = logreturn, times = dates)#, field = mesh.index)
f.s <- logreturn ~   f(field, model = rspde_model) 
rspde_fit <- inla(f.s, family = "Gamma", data = data, 
        control.inla=list(int.strategy='eb'))

## Or try this, code from the inla paper

In [9]:
formula<-logreturn~f(times,model="ar1")
result=inla(formula,family="stochvol.nig",data=data)   
summary(result)

Time used:
    Pre = 0.19, Running = 1.45, Post = 0.0173, Total = 1.65 
Fixed effects:
              mean    sd 0.025quant 0.5quant 0.975quant   mode kld
(Intercept) -8.324 0.075     -8.469   -8.326      -8.17 -8.326   0

Random effects:
  Name	  Model
    times AR1 model

Model hyperparameters:
                                     mean    sd 0.025quant 0.5quant 0.975quant
skewness parameter for stochvol-nig 0.057 0.034     -0.009    0.057      0.124
shape parameter for stochvol-nig    1.015 0.016      1.002    1.010      1.056
Precision for times                 5.859 1.883      3.070    5.559     10.400
Rho for times                       0.698 0.117      0.430    0.713      0.881
                                     mode
skewness parameter for stochvol-nig 0.057
shape parameter for stochvol-nig    1.004
Precision for times                 4.995
Rho for times                       0.744

Marginal log-Likelihood:  6545.25 
 is computed 
Posterior summaries for the linear predictor and